In [1]:
import os
import subprocess
import couchdb
import pandas as pd
# from couchdb_settings import *


username = 'grp5admin'
password = 'password'
address = '172.26.130.201:5984'
tweets = 'raw_tweets'

tweet_db = couchdb.Database('http://' + address + '/' + tweets)
tweet_db.resource.credentials = (username, password)

In [2]:
geo = {}

for item in testdb.view('geo-test/geo-count-historic', group = True):
    key = str(item.key)
    geo[key] = item.value
    
# print(geo)
    

NameError: name 'testdb' is not defined

In [3]:
languages = {}

for item in testdb.view('geo-test/lang-count', group = True, group_level = 1):
    if item.key != 'en':
        if item.key == 'in':
            languages['id'] = item.value
        else:
            languages[item.key] = item.value
            
        languages = {k:v for k, v in sorted(languages.items(), key=lambda item: item[1])[::-1][:20]}

# print(languages)

langCode = {}

with open('Data/langCode.json', 'r', encoding= 'utf-8') as f:
    for line in f:
        (val, key) = line.split()
        langCode[key] = val
        
languages = {v2: v1 for k1, v1 in languages.items() for k2, v2 in langCode.items() if k1 == k2}

print(languages)

NameError: name 'testdb' is not defined

In [4]:
def top_10_birth_country(file, N):
    data = pd.read_csv(file)
    
    match_cols = []
    new_cols = []
    col_names = data.columns
    for name in col_names:
        if name.endswith('_p'):
            match_cols.append(name)
            new_cols.append(name.strip())

    ext_data = pd.DataFrame(data[match_cols])
    ext_data.columns = new_cols
    
    grand_total = ext_data['tot_p'].sum(axis = 0)
    country_total = ext_data.sum(axis = 0)
    percentage = country_total/grand_total * 100
    
    birth_country = pd.DataFrame(country_total, columns = ['country_total'])
    birth_country['percentage'] = percentage
    
    drop_columns = ['hong_kong_sar_china_p', 'born_elsewhere_p', 'tot_p', 'os_visitors_p', 'country_birth_not_stated_p', 'australia_p', 'new_zealand_p', 'united_states_america_p', 'united_kingdom_ci_im_p', 'fiji_p', 'south_africa_p']
    
    birth_country.loc['china_excl_sars_taiwan_p'] += birth_country.loc['hong_kong_sar_china_p']
    birth_country = birth_country.T.drop(drop_columns, axis = 1)
    birth_country = birth_country.rename({'china_excl_sars_taiwan_p' : 'china_p', 'sri_lanka_p' : 'srilanka_p'}, axis = 1)
    
    country_names = []
    for item in birth_country.columns:
        item = item.split('_')
        country_names.append(item[0].capitalize())

    birth_country.columns = country_names
    birth_country = birth_country.rename({'Srilanka' : 'Sri Lanka'}, axis = 1).T
    birth_country = birth_country.sort_values(by = ['country_total'], ascending = False)[:N]
    
    return birth_country

top_10_birth_country('Data/AURIN/country_of_birth.csv', 20)

,country_total,percentage
China,174418.0,3.855451
India,160058.0,3.538028
Vietnam,78036.0,1.724960
Italy,61521.0,1.359901
Sri Lanka,52658.0,1.163987
Malaysia,45852.0,1.013543
Greece,43881.0,0.969975
Philippines,43642.0,0.964692
Pakistan,19127.0,0.422796
Germany,17511.0,0.387075


In [5]:
def top_n_lang_spoken_at_home(file_path, N):
    
    data = pd.read_csv(file_path)

    match_cols = []
    new_cols = []
    col_names = data.columns
    for name in col_names:
        if name.endswith('_P'):
            match_cols.append(name)   
            new_cols.append(name.strip())

    ext_data = pd.DataFrame(data[match_cols])
    ext_data.columns = new_cols

    SOL_tot = ext_data['SOL_Tot_P'].sum(axis = 0)
    tot = ext_data['Total_P'].sum(axis = 0)
    SOL_perc = SOL_tot/tot * 100

    drop_columns = ['SOL_Other_P', 'SOL_Samoan_P', 'SOL_Assyrian_P','SOL_Iran_Lan_Tot_P', 
                    'SOL_Irani_Lan_Othr_P', 'SOL_Se_As_A_L_Othr_P', 'SOL_Aus_Indig_Lang_P', 
                    'SOL_In_Ar_Lang_Othr_P', 'SOL_In_Ar_Lang_Tot_P', 'SOL_Se_As_A_L_Tot_P', 
                    'Language_spoken_home_ns_P', 'SOL_Tot_P', 'Total_P', 
                    'SOL_Chin_lang_Mand_P', 'SOL_Chin_lang_Other_P', 'SOL_Chin_lang_Cant_P']
    
    ext_data = ext_data.drop(drop_columns, axis = 1)
    lang_tot = ext_data.sum(axis = 0)
    
    columns = []
    for index in lang_tot.index:
        idx = index.split('_')
        if 'Se_As' in index or 'In_Ar' in index or 'Ir_Lang' in index:
            columns.append(idx[-2])
        elif 'Ir_La' in index:
            columns.append(idx[3])
        else:
            columns.append(idx[1])

    lang_tot.index = columns
    lang_data = pd.DataFrame(lang_tot, columns = ['number']).T
    lang_data = lang_data.rename({'Pe' : 'Persian'}, axis = 1).T
    lang_data['percentage_SOL'] = lang_data['number']/SOL_tot * 100
    lang_data['percentage_Total'] = lang_data['number']/tot * 100

    langdict = {k:v for v in langCode.values() for k in lang_data.index if k in v}

    idx = []
    for i in lang_data.index:
        name = langdict[i]
        idx.append(name)

    lang_data.index = idx
    lang_data = lang_data.sort_values(by = ['number'], ascending = False)[:N]
    
    spoken = {}
    for i in range(len(lang_data)):
        spoken[lang_data.index[i]] = lang_data.number[i], lang_data.percentage_SOL[i], lang_data.percentage_Total[i]

    return spoken
    
    
top_n_lang_spoken_at_home('Data/AURIN/lang_at_home.csv', 20)

NameError: name 'langCode' is not defined

In [284]:
data = pd.read_csv('country_of_birth.csv')

In [285]:
match_cols = []
new_cols = []

col_names = data.columns

for name in col_names:
    if name.endswith('_p'):
        match_cols.append(name)
        new_cols.append(name.strip())

ext_data = pd.DataFrame(data[match_cols])
ext_data.columns = new_cols

In [286]:
grand_total = ext_data['tot_p'].sum(axis = 0)
country_total = ext_data.sum(axis = 0)
percentage = country_total/grand_total * 100
birth_country = pd.DataFrame(country_total, columns = ['country_total'])
birth_country['percentage'] = percentage

In [287]:
birth_country = birth_country.sort_values(by = ['country_total'], ascending = False)

In [288]:
drop_columns = ['hong_kong_sar_china_p', 'born_elsewhere_p', 'tot_p', 'os_visitors_p', 'country_birth_not_stated_p', 'australia_p', 'new_zealand_p', 'united_states_america_p', 'united_kingdom_ci_im_p', 'fiji_p', 'south_africa_p']

In [289]:
birth_country.loc['china_excl_sars_taiwan_p'] += birth_country.loc['hong_kong_sar_china_p']
birth_country = birth_country.T.drop(drop_columns, axis = 1).T[:10].T
birth_country = birth_country.rename({'china_excl_sars_taiwan_p' : 'china_p', 'sri_lanka_p' : 'srilanka_p'}, axis = 1)

In [290]:
names = []
for item in birth_country.columns:
    item = item.split('_')
    names.append(item[0].capitalize())

birth_country.columns = names

In [300]:
birth_country = birth_country.rename({'Srilanka' : 'Sri Lanka'}, axis = 1).T

In [365]:
birth_country.index

Index(['India', 'China', 'Vietnam', 'Italy', 'Sri Lanka', 'Malaysia', 'Greece',
       'Philippines', 'Pakistan', 'Germany'],
      dtype='object')

In [317]:
results = []

for i in range(len(birth_country)):
    entry = {}
    e = birth_country.iloc[i]
    entry['name'] = e.name
    entry['y'] = e.country_total
    entry['z'] = e.percentage
    results.append(entry)
print(results)

[{'name': 'India', 'y': 160058.0, 'z': 3.5380284230620616}, {'name': 'China', 'y': 174418.0, 'z': 3.85545140819977}, {'name': 'Vietnam', 'y': 78036.0, 'z': 1.7249596147775874}, {'name': 'Italy', 'y': 61521.0, 'z': 1.3599010772045204}, {'name': 'Sri Lanka', 'y': 52658.0, 'z': 1.1639874339402094}, {'name': 'Malaysia', 'y': 45852.0, 'z': 1.0135430859703463}, {'name': 'Greece', 'y': 43881.0, 'z': 0.9699747918403725}, {'name': 'Philippines', 'y': 43642.0, 'z': 0.9646917769763117}, {'name': 'Pakistan', 'y': 19127.0, 'z': 0.42279592177778097}, {'name': 'Germany', 'y': 17511.0, 'z': 0.38707478361743725}]


In [504]:
data = pd.read_csv('Data/AURIN/lang_at_home.csv')

In [505]:
match_cols = []
new_cols = []

col_names = data.columns

for name in col_names:
    if name.endswith('_P'):
        match_cols.append(name)   
        new_cols.append(name.strip())

ext_data = pd.DataFrame(data[match_cols])
ext_data.columns = new_cols

In [506]:
SOL_tot = ext_data['SOL_Tot_P'].sum(axis = 0)
tot = ext_data['Total_P'].sum(axis = 0)
SOL_perc = SOL_tot/tot * 100
SOL_perc

29.072123556525113

In [507]:
drop_columns = ['SOL_Other_P', 'SOL_Samoan_P', 'SOL_Assyrian_P','SOL_Iran_Lan_Tot_P', 'SOL_Irani_Lan_Othr_P', 'SOL_Se_As_A_L_Othr_P', 'SOL_Aus_Indig_Lang_P', 'SOL_In_Ar_Lang_Othr_P', 'SOL_In_Ar_Lang_Tot_P', 'SOL_Se_As_A_L_Tot_P', 'Language_spoken_home_ns_P', 'SOL_Tot_P', 'Total_P', 'SOL_Chin_lang_Mand_P', 'SOL_Chin_lang_Other_P', 'SOL_Chin_lang_Cant_P']

In [508]:
ext_data = ext_data.drop(drop_columns, axis = 1)

In [509]:
columns = []
lang_tot = ext_data.sum(axis = 0)
for index in lang_tot.index:
    idx = index.split('_')
    if 'Se_As' in index or 'In_Ar' in index or 'Ir_Lang' in index:
        columns.append(idx[-2])
    elif 'Ir_La' in index:
        columns.append(idx[3])
    else:
        columns.append(idx[1])

In [510]:
lang_data = pd.DataFrame(lang_tot, columns = ['count']).T
lang_data.columns = columns

In [511]:
lang_data = lang_data.rename({'Pe' : 'Persian'}, axis = 1).T
lang_data['percentage'] = lang_data['count']/SOL_tot * 100

In [519]:
langCode = {k:v for v in langCode.values() for k in lang_data.index if k in v}

In [6]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

hashtags = {}

for item in testdb.view('geo-test/hash-tags', group = True, group_level = 1):
    if item.key.lower() not in hashtags.keys():
        hashtags[item.key.lower()] = item.value
    else:
        hashtags[item.key.lower()] += item.value

hashtags = {k: v for k, v in sorted(hashtags.items(), key=lambda item: item[1])[-20:]}

hashtags

# string = ','.join(hashtags)

# wc = WordCloud(background_color="white", max_words=100)

# plt.imshow(wc.generate(string), interpolation='bilinear')
# plt.axis("off")
# plt.show()


NameError: name 'testdb' is not defined

In [8]:
import nltk
import re
import string
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import TweetTokenizer

def topic_switch(topic):
    """
    params: topic of selection
    return: paths to views relating to the selected topic 
    """

    if topic == 'housing':
        count_view = 'text/housing-count'
        topic_view = 'text/housing'
        topic_db_text = db_connect('housing_text')
    if topic == 'transportation':
        count_view = 'text/transportation-count'
        topic_view = 'text/transportation'
        topic_db_text = db_connect('trans_text')
    if topic == 'cost':
        count_view = 'text/cost-count'
        topic_view = 'text/cost'
        topic_db_text = db_connect('cost_text')

    return count_view, topic_view, topic_db_text

# def trend_per_topic(db, topic):

#     #db = testdb
#     count_view, _ = topic_switch(topic)
    
#     year_topic = {}
#     year_total = {}
#     percent = {}

#     for item in db.view(count_view, group = True, group_level = 1):
#         year_topic[item.key] = item.value

#     for item in db.view('geo-test/by-year-count', group = True, group_level = 1):
#         year_total[item.key] = item.value

#     for key in year_topic.keys():
#         if key in year_total.keys():
#             percent[key] = year_topic[key]/year_total[key] * 100
            
#     return year_topic, year_total, percent

def delete_docs(topic, save_db):
    docs = []    
    for row in save_db.view(topic + '/all', include_docs=True):
        doc = row['doc']
        if int(doc['year'] >=2018):
            doc['_deleted']=True
            docs.append(doc)
        save_db.update(docs)    

def topic_wordcloud(query_db, topic):
    """
    Extract topic related wordcloud
    params: raw_tweets database;
            topic of selection
    return: corpus of combined tweets on the selected topic indexed by year; 
            all words in lowercases
    return type: dict - {year : corpus as a list}
    render: wordcloud
    """

    _, topic_view, save_db = topic_switch(topic)
    
    delete_docs(topic, save_db)

    yearly_tweets = defaultdict(list)
    for item in query_db.view(topic_view):
        yearly_tweets[item.key].append(item.value)
    
    tokenizer = TweetTokenizer()
    
    for key, tweet in yearly_tweets.items():
        tweet = [' '.join(re.sub("(@[A-Za-z0-9\_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in tweet]
        tweet = [' '.join(tweet)]
        tweet_tokens = tokenizer.tokenize(tweet[0])
        tweet_clean = []
        for word in tweet_tokens:
            if word.lower() not in stopwords.words('english') and word.lower() not in string.punctuation:
                tweet_clean.append(word.lower())
                
        yearly_tweets[key] = ' '.join(tweet_clean)
        
    
    for k, v in yearly_tweets.items():
        save_db.save({'year': k, 'text':v})
            


def db_connect(dbname):
    """
    """

    couchserver = couchdb.Server('http://' + username + ':' + password + '@' + address)
    try:
      db = couchserver[dbname]
    except:
      db = couchserver.create(dbname)

    return db


for topic in ['cost', 'housing', 'transportation']:
    topic_wordcloud(tweet_db, topic)
     

In [ ]:
trend_per_topic(testdb, 'housing')

({2015: 8}, {2015: 10623, 2018: 3, 2022: 3}, {2015: 0.07530829332580251})

In [9]:
from textblob import TextBlob

def topic_sentiment(topic):

    _, _, db = topic_switch(topic)
    
    yearly_tweets = {}
    for item in db.view(topic + '/text'):
        yearly_tweets[item.key] = item.value
    
    yearly_sentiment = {}
    for key, value in yearly_tweets.items():
        blob = TextBlob(value)
        for sentence in blob.sentences:
            sentiment = sentence.sentiment.polarity
            yearly_sentiment[key] = sentiment
            
    return yearly_sentiment

topic_sentiment('transportation')

{2018: 0.26719977553310886,
 2019: 0.05319404621352675,
 2020: 0.08493145743145744,
 2021: 0.06922077922077922,
 2022: -0.014355069812816458}